In [1]:
pip install kagglehub


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import kagglehub

# Download the latest version of the dataset
dataset_path = kagglehub.dataset_download("kazireyazulhasan/19000-skincare-products-database-of-skinsort")

# Specify the directory where you want to store the dataset
destination_path = r'C:\Users\ahmed\OneDrive\Desktop\DATASET'

print("Path to downloaded dataset files:", dataset_path)
print("Intended destination path for organization:", destination_path)


Path to downloaded dataset files: C:\Users\ahmed\.cache\kagglehub\datasets\kazireyazulhasan\19000-skincare-products-database-of-skinsort\versions\1
Intended destination path for organization: C:\Users\ahmed\OneDrive\Desktop\DATASET


In [1]:
pip install pymongo flask


Defaulting to user installation because normal site-packages is not writeable
                                              0.0/876.5 kB ? eta -:--:--
     -                                     30.7/876.5 kB 640.0 kB/s eta 0:00:02
     -                                     30.7/876.5 kB 640.0 kB/s eta 0:00:02
     --                                    61.4/876.5 kB 465.5 kB/s eta 0:00:02
     ----                                 112.6/876.5 kB 595.3 kB/s eta 0:00:02
     -----                                143.4/876.5 kB 607.9 kB/s eta 0:00:02
     -------                              194.6/876.5 kB 692.9 kB/s eta 0:00:01
     ---------                            225.3/876.5 kB 687.0 kB/s eta 0:00:01
     ----------                           256.0/876.5 kB 714.4 kB/s eta 0:00:01
     -----------                          286.7/876.5 kB 707.1 kB/s eta 0:00:01
     -------------                        337.9/876.5 kB 748.1 kB/s eta 0:00:01
     -------------                        337.9/8


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import os

# List directory path
downloaded_path = "C:\\Users\\ahmed\\OneDrive\\Desktop\\DATASET"
csv_file_path = os.path.join(downloaded_path, 'datasheet.csv')  # Update the file name as needed

# Load CSV
df = pd.read_csv(csv_file_path)

# Inspect the first few rows
print(df.head())

# Filter only necessary fields
df = df[['Name', 'Brand', 'Type', 'Ingredients', 'SkinType', 'Concerns', 'Usage', 'Allergens', 'Rating', 'ImageURL']]

# Clean missing data (if any)
df.dropna(subset=['Name', 'Brand'], inplace=True)

# Verify cleaned data
print(df.head())


                 brand                                      name  \
0         The Ordinary          Glycolic Acid 7% Toning Solution   
1       La Roche-Posay  Toleriane Hydrating Gentle Face Cleanser   
2         The Ordinary                Niacinamide 10% + Zinc 1%    
3  Youth To The People            Superfood Antioxidant Cleanser   
4                COSRX          Low pH Good Morning Gel Cleanser   

               type        country  \
0             Toner         Canada   
1     Face Cleanser         France   
2  Facial Treatment         Canada   
3     Face Cleanser  United States   
4     Face Cleanser    South Korea   

                                         ingridients  \
0  Water,Glycolic Acid,Rosa Damascena Flower Wate...   
1  Water,Glycerin,Pentaerythrityl Tetraethylhexan...   
2  Water,Niacinamide,Pentylene Glycol,Zinc PCA,Di...   
3  Water,Cocamidopropyl Hydroxysultaine,Sodium Co...   
4  Water,Cocamidopropyl Betaine,Sodium Lauroyl Me...   

                         

KeyError: "None of [Index(['Name', 'Brand', 'Type', 'Ingredients', 'SkinType', 'Concerns', 'Usage',\n       'Allergens', 'Rating', 'ImageURL'],\n      dtype='object')] are in the [columns]"

In [4]:
from pymongo import MongoClient

# Establish connection
client = MongoClient("mongodb://localhost:27017/")

# Select database and collection
db = client['skincare_database']
products = db['products']

# Convert DataFrame to JSON-like records
records = df.to_dict(orient='records')

# Insert data into MongoDB
insert_result = products.insert_many(records)

# Confirm successful insert
print(f"Inserted {len(insert_result.inserted_ids)} records into MongoDB.")


Inserted 19050 records into MongoDB.


In [6]:
#WEB SCRAPING FOR IMAGES 
!pip install requests beautifulsoup4


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import requests
from bs4 import BeautifulSoup
import re

# Function to scrape Google or other image search results
def search_images(product_name):
    # Google Image search URL with query
    search_query = '+'.join(product_name.split())
    url = f"https://www.google.com/search?tbm=isch&q={search_query}"
    
    # Set User-Agent to avoid being blocked by Google
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    # Send request
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        # Parse with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract image URLs from the page
        img_tags = soup.find_all('img')
        image_urls = []
        for img in img_tags:
            try:
                img_url = img['src']
                # Regex to ensure it's a valid URL
                if re.match(r'^https?://.*\.(jpg|jpeg|png)$', img_url):
                    image_urls.append(img_url)
            except KeyError:
                continue
        
        # Return the first image URL (you can adjust logic to fetch more if needed)
        if image_urls:
            return image_urls[0]
    return None


In [10]:
from pymongo import MongoClient

# Establish MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
db = client['skincare_database']
products = db['products']

# Function to simulate image search
def search_images(query):
    # Replace this placeholder with actual web scraping or API integration logic
    # Simulated image URL for demonstration purposes
    return f"https://example.com/images/{query.replace(' ', '_')}.jpg"

# Loop through database entries and enrich with scraped image links
for product in products.find():
    if not product.get('image_url'):  # Only scrape if no image exists already
        # Combine brand and name for a more specific search query
        search_query = f"{product['brand']} {product['name']}"
        image_url = search_images(search_query)
        
        if image_url:
            # Update MongoDB with the image URL
            products.update_one(
                {"_id": product["_id"]},
                {"$set": {"image_url": image_url}}
            )
            print(f"Added image for product: {search_query}")
        else:
            print(f"No image found for: {search_query}")


Added image for product: Paula's Choice Anti-Aging Lip Gloss SPF 40
Added image for product: Neogen Vita Duo Night Cream
Added image for product: Make Up For Ever Artist Color Pencil Brow, Eye & Lip Liner - 606 Wherever Walnut
Added image for product: Cell:Monde Nucleic Acid Refreshing Toner
Added image for product: Bali Body Antioxidant Body Whip
Added image for product: Bee Bald Clean Daily Cleanser
Added image for product: I'm from Honey Glow Cream
Added image for product: SVR Sebiaclear Serum
Added image for product: Essence Lash Princess Curl & Volume Mascara
Added image for product: Drunk Elephant Slaai™ Makeup-Melting Butter Cleanser
Added image for product: Skin Gym R-Dew Retinol Body Lotion
Added image for product: Dermacol Sensitive Micellar Lotion
Added image for product: Pyunkang Yul Calming Moisture Serum
Added image for product: Mac Cosmetics Studio Finish Concealer
Added image for product: One Thing Citrus Unshiu Fruit Extract
Added image for product: A'Pieu Madecassosid

In [14]:
pip install selenium


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

                                              0.0/9.7 MB ? eta -:--:--
                                              0.0/9.7 MB 1.3 MB/s eta 0:00:08
                                              0.0/9.7 MB 1.3 MB/s eta 0:00:08
                                              0.1/9.7 MB 465.5 kB/s eta 0:00:21
                                              0.1/9.7 MB 525.1 kB/s eta 0:00:19
                                              0.1/9.7 MB 599.1 kB/s eta 0:00:16
                                              0.2/9.7 MB 610.0 kB/s eta 0:00:16
                                              0.2/9.7 MB 692.9 kB/s eta 0:00:14
                                              0.2/9.7 MB 687.0 kB/s eta 0:00:14
                                              0.2/9.7 MB 687.0 kB/s eta 0:00:14
     -                                        0.3/9.7 MB 728.0 kB/s e


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
pip install undetected-chromedriver


Defaulting to user installation because normal site-packages is not writeable
                                              0.0/65.4 kB ? eta -:--:--
     ------------------                       30.7/65.4 kB 1.4 MB/s eta 0:00:01
     ------------------                       30.7/65.4 kB 1.4 MB/s eta 0:00:01
     -----------------------                41.0/65.4 kB 281.8 kB/s eta 0:00:01
     -------------------------------------- 65.4/65.4 kB 394.1 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/163.3 kB ? eta -:--:--
     -------                                 30.7/163.3 kB 1.3 MB/s eta 0:00:01
     -------------                         61.4/163.3 kB 812.7 kB/s eta 0:00:01
     ------------------------             112.6/163.3 kB 819.2 kB/s eta 0:00:01
     -------------------------------      143.4/163.3 kB 774.0 kB/s eta 0:00:01
     ------------------------------


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pymongo import MongoClient
import time

# MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
db = client['skincare_database']
products = db['products']

# Selenium setup with undetected-chromedriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run browser in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

driver = uc.Chrome(options=chrome_options)

# Function to scrape price and reviews
def scrape_product_data(brand, product_name):
    search_query = f"{brand} {product_name}".replace(" ", "+")
    url = f"https://www.amazon.com/s?k={search_query}"

    driver.get(url)
    time.sleep(3)  # Allow page to load

    try:
        # Scrape price
        price_element = driver.find_element(By.CLASS_NAME, "a-price-whole")
        price = price_element.text

        # Scrape reviews
        review_elements = driver.find_elements(By.CSS_SELECTOR, ".a-size-base.review-text.review-text-content")
        reviews = [review.text.strip() for review in review_elements[:5]]  # Limit to 5 reviews
    except Exception as e:
        print(f"Error while scraping: {e}")
        price, reviews = None, None

    return price, reviews

# Update MongoDB with scraped data
for product in products.find():
    product_name = product['name']
    brand_name = product['brand']

    print(f"Scraping data for: {brand_name} {product_name}")
    price, reviews = scrape_product_data(brand_name, product_name)

    if price or reviews:
        update_data = {}
        if price:
            update_data['price'] = float(price.replace(",", ""))
        if reviews:
            update_data['reviews'] = [{"reviewer": "Anonymous", "text": review} for review in reviews]

        # Update MongoDB
        products.update_one(
            {"_id": product["_id"]},
            {"$set": update_data}
        )
        print(f"Updated product: {product_name} with price and reviews")
    else:
        print(f"No data found for: {brand_name} {product_name}")

driver.quit()


could not detect version_main.therefore, we are assuming it is chrome 108 or higher


Scraping data for: The Ordinary Glycolic Acid 7% Toning Solution
Updated product: Glycolic Acid 7% Toning Solution with price and reviews
Scraping data for: La Roche-Posay Toleriane Hydrating Gentle Face Cleanser
Updated product: Toleriane Hydrating Gentle Face Cleanser with price and reviews
Scraping data for: The Ordinary Niacinamide 10% + Zinc 1% 
Updated product: Niacinamide 10% + Zinc 1%  with price and reviews
Scraping data for: Youth To The People Superfood Antioxidant Cleanser
Updated product: Superfood Antioxidant Cleanser with price and reviews
Scraping data for: COSRX Low pH Good Morning Gel Cleanser
Updated product: Low pH Good Morning Gel Cleanser with price and reviews
Scraping data for: La Roche-Posay Toleriane Purifying Foaming Face Cleanser
Updated product: Toleriane Purifying Foaming Face Cleanser with price and reviews
Scraping data for: The Ordinary Azelaic Acid Suspension 10%
Updated product: Azelaic Acid Suspension 10% with price and reviews
Scraping data for: COS

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00443433+25059]
	(No symbol) [0x003CCE34]
	(No symbol) [0x002ABEC3]
	(No symbol) [0x0029D773]
	(No symbol) [0x0029D481]
	(No symbol) [0x0029C619]
	(No symbol) [0x002B7537]
	(No symbol) [0x0032B854]
	(No symbol) [0x00311C96]
	(No symbol) [0x002E3FAC]
	(No symbol) [0x002E4F3D]
	GetHandleVerifier [0x00735593+3113795]
	GetHandleVerifier [0x0074A25A+3198986]
	GetHandleVerifier [0x00742A32+3168226]
	GetHandleVerifier [0x004E32A0+680016]
	(No symbol) [0x003D577D]
	(No symbol) [0x003D2A28]
	(No symbol) [0x003D2BC5]
	(No symbol) [0x003C5820]
	BaseThreadInitThunk [0x75757BA9+25]
	RtlInitializeExceptionChain [0x7784C0CB+107]
	RtlClearBits [0x7784C04F+191]


In [11]:
pip install flask flask-cors 


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from flask import Flask, request, jsonify
from pymongo import MongoClient
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Enable CORS for frontend-backend communication

# MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
db = client['skincare_database']
products = db['products']

@app.route('/products', methods=['GET'])
def get_products():
    # Fetch all products from MongoDB
    product_list = list(products.find({}, {'_id': 0}))  # Exclude _id for simplicity
    return jsonify(product_list)

@app.route('/products', methods=['POST'])
def add_product():
    # Add a new product
    data = request.json
    products.insert_one(data)
    return jsonify({"message": "Product added successfully!"}), 201

@app.route('/products/<string:name>', methods=['DELETE'])
def delete_product(name):
    # Delete a product by name
    result = products.delete_one({"name": name})
    if result.deleted_count > 0:
        return jsonify({"message": "Product deleted successfully!"})
    else:
        return jsonify({"error": "Product not found"}), 404

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\ahmed\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
